In [0]:
#colabを使う方はこちらを使用ください。
!pip install torch==0.4.1
!pip install torchvision==0.2.1
!pip install numpy==1.14.6
!pip install matplotlib==2.1.2
!pip install pillow==5.0.0
!pip install opencv-python==3.4.3.18

    100% |████████████████████████████████| 519.5MB 24kB/s 
tcmalloc: large alloc 1073750016 bytes == 0x55c871a86000 @  0x7fd8e041b2a4 0x55c817ebdf18 0x55c817fb1a85 0x55c817ed14ca 0x55c817ed6232 0x55c817eced0a 0x55c817ed65fe 0x55c817eced0a 0x55c817ed65fe 0x55c817eced0a 0x55c817ed65fe 0x55c817eced0a 0x55c817ed6c38 0x55c817eced0a 0x55c817ed65fe 0x55c817eced0a 0x55c817ed65fe 0x55c817ed6232 0x55c817ed6232 0x55c817eced0a 0x55c817ed6c38 0x55c817ed6232 0x55c817eced0a 0x55c817ed6c38 0x55c817eced0a 0x55c817ed6c38 0x55c817eced0a 0x55c817ed65fe 0x55c817eced0a 0x55c817ece629 0x55c817eff61f
  Using cached https://files.pythonhosted.org/packages/ca/0d/f00b2885711e08bd71242ebe7b96561e6f6d01fdb4b9dcf4d37e2e13c5e1/torchvision-0.2.1-py2.py3-none-any.whl
    100% |████████████████████████████████| 2.0MB 12.6MB/s 
  Found existing installation: Pillow 4.0.0
    Uninstalling Pillow-4.0.0:
      Successfully uninstalled Pillow-4.0.0
    100% |████████████████████████████████| 5.9MB 5.3MB/s 
  Found existing

In [1]:
#colabを使う方はこちらを使用ください。
#Google Driveにマウント
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
#colabを使う方はこちらを使用ください。※変更の必要がある場合はパスを変更してください。
cd /content/gdrive/My Drive/Colab Notebooks/pytorch_handbook/chapter6/

/content/gdrive/My Drive/Colab Notebooks/pytorch_handbook/chapter6


In [3]:
#colabを使う方はこちらを使用ください。
!ls

net.py	     result_cgan   section6_3.ipynb  train_cgan.py
__pycache__  result_lsgan  section6_4.ipynb  train_lsgan.py


In [0]:
import os
import random
import numpy as np

import torch.nn as nn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils

from net import weights_init, Generator, Discriminator

In [0]:
def onehot_encode(label, device, n_class=10):
    """
    カテゴリカル変数のラベルをOne-Hoe形式に変換する
    :param label: 変換対象のラベル
    :param device: 学習に使用するデバイス。CPUあるいはGPU
    :param n_class: ラベルのクラス数
    :return:
    """
    eye = torch.eye(n_class, device=device)
    # ランダムベクトルあるいは画像と連結するために(B, c_class, 1, 1)のTensorにして戻す
    return eye[label].view(-1, n_class, 1, 1)   

In [0]:
def concat_image_label(image, label, device, n_class=10):
    """
    画像とラベルを連結する
    :param image:　画像
    :param label: ラベル
    :param device: 学習に使用するデバイス。CPUあるいはGPU
    :param n_class: ラベルのクラス数
    :return:　画像とラベルをチャネル方向に連結したTensor
    """
    B, C, H, W = image.shape    # 画像Tensorの大きさを取得
    
    oh_label = onehot_encode(label, device)         # ラベルをOne-Hotベクトル化
    oh_label = oh_label.expand(B, n_class, H, W)    # 画像のサイズに合わせるようラベルを拡張する
    return torch.cat((image, oh_label), dim=1)      # 画像とラベルをチャネル方向（dim=1）で連結する

In [0]:
def concat_noise_label(noise, label, device):
    """
    ノイズ（ランダムベクトル）とラベルを連結する
    :param noise: ノイズ
    :param label: ラベル
    :param device: 学習に使用するデバイス。CPUあるいはGPU
    :return:　ノイズとラベルを連結したTensor
    """
    oh_label = onehot_encode(label, device)     # ラベルをOne-Hotベクトル化
    return torch.cat((noise, oh_label), dim=1)  # ノイズとラベルをチャネル方向（dim=1）で連結する

In [8]:
workers = 2
batch_size = 50
nz = 100
nch_g = 64
nch_d = 64
n_epoch = 200
lr = 0.0002
beta1 = 0.5
outf = './result_cgan'
display_interval = 100

try:
    os.makedirs(outf)
except OSError:
    pass

random.seed(0)
np.random.seed(0)
torch.manual_seed(0)

In [9]:
trainset = dset.STL10(root='../../dataset/stl10_root', download=True, split='train',
                      transform=transforms.Compose([
                          transforms.RandomResizedCrop(64, scale=(88/96, 1.0), ratio=(1., 1.)),
                          transforms.RandomHorizontalFlip(),
                          transforms.ColorJitter(brightness=0.05, contrast=0.05, saturation=0.05, hue=0.05),
                          transforms.ToTensor(),
                          transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                      ]))   # ラベルを使用するのでunlabeledを含めない
testset = dset.STL10(root='../../dataset/stl10_root', download=True, split='test',
                     transform=transforms.Compose([
                         transforms.RandomResizedCrop(64, scale=(88/96, 1.0), ratio=(1., 1.)),
                         transforms.RandomHorizontalFlip(),
                         transforms.ColorJitter(brightness=0.05, contrast=0.05, saturation=0.05, hue=0.05),
                         transforms.ToTensor(),
                         transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                     ]))
dataset = trainset + testset

dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size,
                                         shuffle=True, num_workers=int(workers))

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('device:', device)

Files already downloaded and verified
Files already downloaded and verified
device: cuda:0


In [10]:
# 生成器G。ランダムベクトルとラベルを連結したベクトルから贋作画像を生成する
netG = Generator(nz=nz+10, nch_g=nch_g).to(device)   # 入力ベクトルの次元は、ランダムベクトルの次元nzにクラス数10を加算したもの
netG.apply(weights_init)
print(netG)

Generator(
  (layers): ModuleDict(
    (layer0): Sequential(
      (0): ConvTranspose2d(110, 512, kernel_size=(4, 4), stride=(1, 1))
      (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
    (layer1): Sequential(
      (0): ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
      (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
    (layer2): Sequential(
      (0): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
      (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
    (layer3): Sequential(
      (0): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
    (layer4): Sequential(
      (0): ConvTranspose2d(64, 3, ker

In [11]:
# 識別器D。画像とラベルを連結したTensorが、元画像か贋作画像かを識別する
netD = Discriminator(nch=3+10, nch_d=nch_d).to(device)   # 入力Tensorのチャネル数は、画像のチャネル数3にクラス数10を加算したもの
netD.apply(weights_init)
print(netD)

Discriminator(
  (layers): ModuleDict(
    (layer0): Sequential(
      (0): Conv2d(13, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
      (1): LeakyReLU(negative_slope=0.2)
    )
    (layer1): Sequential(
      (0): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
      (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.2)
    )
    (layer2): Sequential(
      (0): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
      (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.2)
    )
    (layer3): Sequential(
      (0): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
      (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.2)
    )
    (layer4): Conv2d(512, 1, kernel_size=(4, 4), stride=(1, 1))
  )
)


In [0]:
criterion = nn.MSELoss()

optimizerD = optim.Adam(netD.parameters(), lr=lr, betas=(beta1, 0.999), weight_decay=1e-5)
optimizerG = optim.Adam(netG.parameters(), lr=lr, betas=(beta1, 0.999), weight_decay=1e-5)

fixed_noise = torch.randn(batch_size, nz, 1, 1, device=device)

fixed_label = [i for i in range(10)] * (batch_size // 10)  # 確認用のラベル。0〜9のラベルの繰り返し
fixed_label = torch.tensor(fixed_label, dtype=torch.long, device=device)

fixed_noise_label = concat_noise_label(fixed_noise, fixed_label, device)  # 確認用のノイズとラベルを連結

In [13]:
# 学習のループ
for epoch in range(n_epoch):
    for itr, data in enumerate(dataloader):
        real_image = data[0].to(device)     # 元画像
        real_label = data[1].to(device)     # 元画像に対応するラベル
        real_image_label = concat_image_label(real_image, real_label, device)   # 元画像とラベルを連結

        sample_size = real_image.size(0)
        noise = torch.randn(sample_size, nz, 1, 1, device=device)
        fake_label = torch.randint(10, (sample_size,), dtype=torch.long, device=device)     # 贋作画像生成用のラベル
        fake_noise_label = concat_noise_label(noise, fake_label, device)    # ノイズとラベルを連結
        
        real_target = torch.full((sample_size,), 1., device=device)
        fake_target = torch.full((sample_size,), 0., device=device)

        ############################
        # 識別器Dの更新
        ###########################
        netD.zero_grad()

        output = netD(real_image_label)     # 識別器Dで元画像とラベルの組み合わせに対する識別信号を出力
        errD_real = criterion(output, real_target)        
        D_x = output.mean().item()

        fake_image = netG(fake_noise_label)     # 生成器Gでラベルに対応した贋作画像を生成
        fake_image_label = concat_image_label(fake_image, fake_label, device)   # 贋作画像とラベルを連結

        output = netD(fake_image_label.detach())    # 識別器Dで贋作画像とラベルの組み合わせに対する識別信号を出力
        errD_fake = criterion(output, fake_target)
        D_G_z1 = output.mean().item()

        errD = errD_real + errD_fake
        errD.backward()
        optimizerD.step()

        ############################
        # 生成器Gの更新
        ###########################
        netG.zero_grad()
        
        output = netD(fake_image_label)     # 更新した識別器Dで改めて贋作画像とラベルの組み合わせに対する識別信号を出力
        errG = criterion(output, real_target)
        errG.backward()
        D_G_z2 = output.mean().item()
        
        optimizerG.step()

        if itr % display_interval == 0:
            print('[{}/{}][{}/{}] Loss_D: {:.3f} Loss_G: {:.3f} D(x): {:.3f} D(G(z)): {:.3f}/{:.3f}'
                  .format(epoch + 1, n_epoch,
                          itr + 1, len(dataloader),
                          errD.item(), errG.item(), D_x, D_G_z1, D_G_z2))

        if epoch == 0 and itr == 0:
            vutils.save_image(real_image, '{}/real_samples.png'.format(outf),
                              normalize=True, nrow=10)

    ############################
    # 確認用画像の生成
    ############################
    fake_image = netG(fixed_noise_label)    # 1エポック終了ごとに、指定したラベルに対応する贋作画像を生成する
    vutils.save_image(fake_image.detach(), '{}/fake_samples_epoch_{:03d}.png'.format(outf, epoch + 1),
                      normalize=True, nrow=10)

    ############################
    # モデルの保存
    ############################
    if (epoch + 1) % 50 == 0:
        torch.save(netG.state_dict(), '{}/netG_epoch_{}.pth'.format(outf, epoch + 1))
        torch.save(netD.state_dict(), '{}/netD_epoch_{}.pth'.format(outf, epoch + 1))

[1/200][1/260] Loss_D: 1.958 Loss_G: 31.280 D(x): 0.480 D(G(z)): 0.105/1.759
[1/200][101/260] Loss_D: 0.378 Loss_G: 0.744 D(x): 0.690 D(G(z)): -0.108/0.208
[1/200][201/260] Loss_D: 0.394 Loss_G: 0.652 D(x): 0.660 D(G(z)): 0.315/0.230
[2/200][1/260] Loss_D: 0.574 Loss_G: 1.132 D(x): 0.641 D(G(z)): 0.362/-0.007
[2/200][101/260] Loss_D: 0.636 Loss_G: 0.553 D(x): 0.544 D(G(z)): 0.403/0.314
[2/200][201/260] Loss_D: 0.093 Loss_G: 1.085 D(x): 0.985 D(G(z)): 0.098/-0.027
[3/200][1/260] Loss_D: 0.380 Loss_G: 0.811 D(x): 0.604 D(G(z)): 0.200/0.157
[3/200][101/260] Loss_D: 0.493 Loss_G: 0.995 D(x): 1.009 D(G(z)): 0.620/0.023
[3/200][201/260] Loss_D: 0.486 Loss_G: 0.848 D(x): 0.500 D(G(z)): 0.382/0.087
[4/200][1/260] Loss_D: 0.330 Loss_G: 0.462 D(x): 0.572 D(G(z)): 0.242/0.342
[4/200][101/260] Loss_D: 0.656 Loss_G: 1.184 D(x): 0.932 D(G(z)): 0.707/-0.054
[4/200][201/260] Loss_D: 0.496 Loss_G: 0.732 D(x): 0.848 D(G(z)): 0.571/0.160
[5/200][1/260] Loss_D: 0.814 Loss_G: 0.474 D(x): 0.581 D(G(z)): 0.6

Process Process-77:
Process Process-78:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/dataloader.py", line 96, in _worker_loop
    r = index_queue.get(timeout=MANAGER_STATUS_CHECK_INTERVAL)
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/dataloader.py", line 106, in _worker_loop
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/usr/lib/python3.6/multiprocessing/queues.py", line 104, in get
    if not self._poll(timeout):
  File "/usr/local/lib/p

KeyboardInterrupt: ignored